# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bulayevo,54.9056,70.4439,57.81,68,100,13.44,KZ,1693708419
1,1,bethel,41.3712,-73.4140,68.65,80,0,5.75,US,1693708401
2,2,saint-francois,46.4154,3.9054,58.44,88,0,3.44,FR,1693708420
3,3,port alfred,-33.5906,26.8910,59.85,69,100,6.40,ZA,1693708420
4,4,papatowai,-46.5619,169.4708,48.18,87,100,4.54,NZ,1693708420


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    color = "City",
    geo = True,
    tiles = "OSM"
)


# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.copy()
ideal_weather_df = ideal_weather_df[ideal_weather_df['Cloudiness'] == 0]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Wind Speed'] <10 ]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Humidity'] <50 ]
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Max Temp'] >=50 ) & (ideal_weather_df['Max Temp'] <=85 )]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,new norfolk,-42.7826,147.0587,60.93,33,0,2.93,AU,1693708427
60,60,kharan,28.5833,65.4167,84.88,47,0,5.06,PK,1693708440
96,96,shindand,33.3029,62.1474,74.07,13,0,7.11,AF,1693708455
137,137,petropavlovsk-kamchatsky,53.0452,158.6483,73.99,43,0,2.24,RU,1693708469
155,155,andkhoy,36.9529,65.1238,74.25,18,0,4.90,AF,1693708475
159,159,siwa oasis,29.2041,25.5195,77.14,48,0,4.52,EG,1693708476
197,197,crane,31.3974,-102.3501,84.92,24,0,9.64,US,1693708490
207,207,al jawf,29.5000,38.7500,75.31,39,0,7.47,SA,1693708493
220,220,fallon,39.4735,-118.7774,73.27,49,0,3.44,US,1693708497
232,232,at taj,24.2049,23.2857,79.16,34,0,8.79,LY,1693708240


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,new norfolk,AU,-42.7826,147.0587,33,
60,kharan,PK,28.5833,65.4167,47,
96,shindand,AF,33.3029,62.1474,13,
137,petropavlovsk-kamchatsky,RU,53.0452,158.6483,43,
155,andkhoy,AF,36.9529,65.1238,18,
159,siwa oasis,EG,29.2041,25.5195,48,
197,crane,US,31.3974,-102.3501,24,
207,al jawf,SA,29.5000,38.7500,39,
220,fallon,US,39.4735,-118.7774,49,
232,at taj,LY,24.2049,23.2857,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "format":"json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
new norfolk - nearest hotel: No hotel found
kharan - nearest hotel: No hotel found
shindand - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
andkhoy - nearest hotel: No hotel found
siwa oasis - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
fallon - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
miles city - nearest hotel: No hotel found
swift current - nearest hotel: No hotel found
gillette - nearest hotel: No hotel found
zvishavane - nearest hotel: No hotel found
mead - nearest hotel: No hotel found
annau - nearest hotel: No hotel found
luena - nearest hotel: No hotel found
limulunga - nearest hotel: No hotel found
barra - nearest hotel: No hotel found
portland - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
21,new norfolk,AU,-42.7826,147.0587,33,No hotel found
60,kharan,PK,28.5833,65.4167,47,No hotel found
96,shindand,AF,33.3029,62.1474,13,No hotel found
137,petropavlovsk-kamchatsky,RU,53.0452,158.6483,43,No hotel found
155,andkhoy,AF,36.9529,65.1238,18,No hotel found
159,siwa oasis,EG,29.2041,25.5195,48,No hotel found
197,crane,US,31.3974,-102.3501,24,No hotel found
207,al jawf,SA,29.5000,38.7500,39,No hotel found
220,fallon,US,39.4735,-118.7774,49,No hotel found
232,at taj,LY,24.2049,23.2857,34,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    color = "City",
    geo = True,
    tiles = "OSM",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)